In [1]:
import pandas as pd
import os

def load_datasets(base_path='e:\\SEMESTER 5\\DEEP LEARNING\\FINAL\\neurips-open-polymer-prediction-2025'):
    """
    Load all datasets from the project directory.
    
    Parameters:
    -----------
    base_path : str
        Base path to the project directory
        
    Returns:
    --------
    dict : Dictionary containing all loaded datasets
    """
    datasets = {}
    
    # Load main train dataset
    train_path = os.path.join(base_path, 'train.csv')
    datasets['train'] = pd.read_csv(train_path)
    # Load main test dataset
    test_path = os.path.join(base_path, 'test.csv')
    datasets['test'] = pd.read_csv(test_path)
    
    # Load supplementary datasets
    supplement_path = os.path.join(base_path, 'train_supplement')
    for i in range(1, 5):
        dataset_path = os.path.join(supplement_path, f'dataset{i}.csv')
        datasets[f'dataset{i}'] = pd.read_csv(dataset_path)
    
    return datasets


data = load_datasets()
test_df = data['test']
train_df = data['train']
dataset1_df = data['dataset1']
dataset2_df = data['dataset2']
dataset3_df = data['dataset3']
dataset4_df = data['dataset4']

In [2]:
from rdkit import Chem 
from rdkit.Chem import AllChem 
from rdkit.Chem.rdmolops import RemoveHs

def safe_mol_from_smiles(smiles):
    if pd.isna(smiles) or not isinstance(smiles, str) or smiles.strip() == '':
        return None
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol
    except:
        return None
    
def largest_fragment(mol):
    frags = Chem.GetMolFrags(mol, asMols=True)
    if len(frags) == 1:
        return mol
    largest = max(frags, key=lambda m: m.GetNumAtoms())
    return largest

def neutralize_molecule(mol):
    try:
        # RDKit built-in neutralization
        uncharger = Chem.rdMolStandardize.Uncharger()
        mol = uncharger.uncharge(mol)
        return mol
    except:
        return mol
    
def canonicalize(mol):
    try:
        return Chem.MolToSmiles(mol, canonical=True)
    except:
        return None
    
    
def sanitize_smiles_pipeline(smiles):
    # Step 1: Safe Mol
    mol = safe_mol_from_smiles(smiles)
    if mol is None:
        return None

    # Step 2: Remove salts
    mol = largest_fragment(mol)

    # Step 3: Neutralize
    mol = neutralize_molecule(mol)

    # Step 4: Re-sanitize
    try:
        Chem.SanitizeMol(mol)
    except:
        return None

    # Step 5: Canonicalize
    return canonicalize(mol)

In [8]:
# Replace your validate_smiles_entries call with this:

def validate_smiles_entries_dict(dfs_dict, smiles_column="SMILES"):
    """
    SMILES validation using explicit dictionary keys.
    """
    results = {}

    for df_name, df in dfs_dict.items():
        df_copy = df.copy()

        canonical_list = []
        errors = []

        for s in df_copy[smiles_column]:
            try:
                mol = Chem.MolFromSmiles(s, sanitize=True)
                if mol is None:
                    canonical_list.append(None)
                    errors.append({"SMILES": s, "error": "MolFromSmiles returned None"})
                else:
                    canon = Chem.MolToSmiles(mol, canonical=True)
                    canonical_list.append(canon)
                    errors.append(None)
            except Exception as e:
                canonical_list.append(None)
                errors.append({"SMILES": s, "error": str(e)})

        df_copy["canonical_SMILES"] = canonical_list
        df_copy["sanitization_error"] = errors

        # Summary QC
        total = len(df_copy)
        valid = df_copy["canonical_SMILES"].notna().sum()
        invalid = total - valid
        unique_raw = df_copy[smiles_column].nunique()
        unique_canon = df_copy["canonical_SMILES"].nunique()

        print("\n=====================================")
        print(f"🔎 QC for {df_name}")
        print("=====================================")
        print(f"Total entries:                  {total}")
        print(f"Valid sanitized molecules:      {valid}")
        print(f"Invalid molecules:              {invalid}")
        print(f"Raw unique SMILES:              {unique_raw}")
        print(f"Canonical unique SMILES:        {unique_canon}")
        print(f"Duplicates lost after cleaning: {unique_raw - unique_canon}")
        print("-------------------------------------")

        invalid_df = df_copy[df_copy["canonical_SMILES"].isna()]
        if len(invalid_df) > 0:
            print("❌ Invalid molecules found:")
            print(invalid_df[[smiles_column, "sanitization_error"]].head())
        else:
            print("✅ No invalid molecules.")

        results[df_name] = {
            "clean_df": df_copy[df_copy["canonical_SMILES"].notna()].copy(),
            "invalid_df": invalid_df.copy(),
            "full_df": df_copy,
        }

    return results

# Use explicit dictionary with names
checklist = {
    'test_df': test_df,
    'train_df': train_df,
    'dataset1_df': dataset1_df,
    'dataset2_df': dataset2_df,
    'dataset3_df': dataset3_df,
    'dataset4_df': dataset4_df
}

qc_results = validate_smiles_entries_dict(checklist)

# Now extract cleaned DataFrames
train_df = qc_results['train_df']['clean_df']
dataset1_df = qc_results['dataset1_df']['clean_df']
dataset2_df = qc_results['dataset2_df']['clean_df']
dataset3_df = qc_results['dataset3_df']['clean_df']
dataset4_df = qc_results['dataset4_df']['clean_df']
test_df = qc_results['test_df']['clean_df']


🔎 QC for test_df
Total entries:                  3
Valid sanitized molecules:      3
Invalid molecules:              0
Raw unique SMILES:              3
Canonical unique SMILES:        3
Duplicates lost after cleaning: 0
-------------------------------------
✅ No invalid molecules.

🔎 QC for train_df
Total entries:                  7973
Valid sanitized molecules:      7973
Invalid molecules:              0
Raw unique SMILES:              7973
Canonical unique SMILES:        7973
Duplicates lost after cleaning: 0
-------------------------------------
✅ No invalid molecules.

🔎 QC for dataset1_df
Total entries:                  874
Valid sanitized molecules:      874
Invalid molecules:              0
Raw unique SMILES:              867
Canonical unique SMILES:        866
Duplicates lost after cleaning: 1
-------------------------------------
✅ No invalid molecules.

🔎 QC for dataset2_df
Total entries:                  7208
Valid sanitized molecules:      7208
Invalid molecules:         

In [9]:
dataset_merge = [train_df, dataset1_df, dataset3_df, dataset4_df]
dataset_cluster = [dataset2_df]

def prepare_for_merge(df, key="canonical_SMILES"):
    """
    Remove duplicate SMILES columns and preserve only the key for merging.
    Keeps track of original row count and unique keys.
    """
    df = df.copy()
    
    # Drop raw SMILES if canonical is used for merge
    if "SMILES" in df.columns and key != "SMILES":
        df = df.drop(columns=["SMILES"])

    # Drop duplicate key rows
    before = len(df)
    df = df.drop_duplicates(subset=[key])
    after = len(df)

    print(f"[Prepare] Rows before={before}, after deduplicate={after}")
    return df
from functools import reduce

def robust_merge(datasets, key="canonical_SMILES"):
    """
    Safely merges multiple datasets on canonical SMILES.
    Logs before/after information for traceability.
    """
    # Prepare datasets
    cleaned = [prepare_for_merge(df, key=key) for df in datasets]

    # Log BEFORE merge stats
    print("\n===== BEFORE MERGE STATS =====")
    for i, df in enumerate(cleaned):
        print(f"Dataset {i+1}: rows={len(df)}, unique keys={df[key].nunique()}")

    # Reduce merge
    merged = reduce(
        lambda left, right: left.merge(
            right, on=key, how="outer", suffixes=("", "_dup")
        ),
        cleaned
    )

    # Clean leftover duplicate columns (_dup)
    dup_cols = [c for c in merged.columns if c.endswith("_dup")]
    if dup_cols:
        merged = merged.drop(columns=dup_cols)

    # Log AFTER merge stats
    print("\n===== AFTER MERGE STATS =====")
    print(f"Merged rows={len(merged)}")
    print(f"Merged unique keys={merged[key].nunique()}")

    return merged


def compare_augmentation(base_df, augmented_df, key="canonical_SMILES"):
    """
    Show which molecules the external datasets added.
    """
    base_keys = set(base_df[key])
    aug_keys = set(augmented_df[key])

    added = aug_keys - base_keys
    lost  = base_keys - aug_keys  # should be empty unless outer merge

    print("\n===== AUGMENTATION DIAGNOSTICS =====")
    print(f"New molecules added: {len(added)}")
    print(f"Molecules missing (should be 0): {len(lost)}")

    added_df = augmented_df[augmented_df[key].isin(added)]
    print("Example added rows:")
    print(added_df.head())

    return added_df

df_internal = robust_merge(dataset_merge, key="canonical_SMILES")
df_augmented = robust_merge([df_internal] + dataset_cluster)

augment_inspection = compare_augmentation(df_internal, df_augmented)



[Prepare] Rows before=7973, after deduplicate=7973
[Prepare] Rows before=874, after deduplicate=866
[Prepare] Rows before=46, after deduplicate=46
[Prepare] Rows before=862, after deduplicate=862

===== BEFORE MERGE STATS =====
Dataset 1: rows=7973, unique keys=7973
Dataset 2: rows=866, unique keys=866
Dataset 3: rows=46, unique keys=46
Dataset 4: rows=862, unique keys=862

===== AFTER MERGE STATS =====
Merged rows=8972
Merged unique keys=8972
[Prepare] Rows before=8972, after deduplicate=8972
[Prepare] Rows before=7208, after deduplicate=7174

===== BEFORE MERGE STATS =====
Dataset 1: rows=8972, unique keys=8972
Dataset 2: rows=7174, unique keys=7174

===== AFTER MERGE STATS =====
Merged rows=10343
Merged unique keys=10343

===== AUGMENTATION DIAGNOSTICS =====
New molecules added: 1371
Molecules missing (should be 0): 0
Example added rows:
   id  Tg  FFV  Tc  Density  Rg  \
0 NaN NaN  NaN NaN      NaN NaN   
1 NaN NaN  NaN NaN      NaN NaN   
2 NaN NaN  NaN NaN      NaN NaN   
3 NaN N

In [12]:
df_augmented

,id,Tg,FFV,Tc,Density,Rg,canonical_SMILES,sanitization_error,TC_mean
0,NaN,NaN,NaN,NaN,NaN,NaN,*/C(=C(/*)c1ccc(C(C)(C)C)cc1)c1ccccc1,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,*/C(=C(/*)c1ccc(CCCC)cc1)c1ccccc1,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,*/C(=C(/*)c1ccc(Oc2ccccc2)cc1)c1ccccc1,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,*/C(=C(/*)c1ccc([Si](C(C)C)(C(C)C)C(C)C)cc1)c1...,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,*/C(=C(/*)c1ccc([Si](C)(C)C)cc1)c1ccccc1,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10338,1.115536e+09,NaN,NaN,0.38800,NaN,NaN,*c1sc(*)c(OCCCCCCCCCCCCCCCC)c1C,None,0.38800
10339,2.731393e+08,NaN,NaN,0.37475,0.904355,14.348260,*c1sc(*)c(OCCCCCCCCCCCCCCCCCCCC)c1C,None,0.37475
10340,4.633390e+08,NaN,NaN,0.44475,0.968872,15.087862,*c1sc(*)c2c1OCC(CCCCCCCCCCCCCCCC)O2,None,0.44475
10341,NaN,NaN,NaN,NaN,NaN,NaN,*c1sc(*)c2sc(CCCCCCCCC)nc12,NaN,0.48200


In [14]:
df_augmented = df_augmented.drop(columns=["sanitization_error","id"])
df_augmented

,Tg,FFV,Tc,Density,Rg,canonical_SMILES,TC_mean
0,NaN,NaN,NaN,NaN,NaN,*/C(=C(/*)c1ccc(C(C)(C)C)cc1)c1ccccc1,NaN
1,NaN,NaN,NaN,NaN,NaN,*/C(=C(/*)c1ccc(CCCC)cc1)c1ccccc1,NaN
2,NaN,NaN,NaN,NaN,NaN,*/C(=C(/*)c1ccc(Oc2ccccc2)cc1)c1ccccc1,NaN
3,NaN,NaN,NaN,NaN,NaN,*/C(=C(/*)c1ccc([Si](C(C)C)(C(C)C)C(C)C)cc1)c1...,NaN
4,NaN,NaN,NaN,NaN,NaN,*/C(=C(/*)c1ccc([Si](C)(C)C)cc1)c1ccccc1,NaN
...,...,...,...,...,...,...,...
10338,NaN,NaN,0.38800,NaN,NaN,*c1sc(*)c(OCCCCCCCCCCCCCCCC)c1C,0.38800
10339,NaN,NaN,0.37475,0.904355,14.348260,*c1sc(*)c(OCCCCCCCCCCCCCCCCCCCC)c1C,0.37475
10340,NaN,NaN,0.44475,0.968872,15.087862,*c1sc(*)c2c1OCC(CCCCCCCCCCCCCCCC)O2,0.44475
10341,NaN,NaN,NaN,NaN,NaN,*c1sc(*)c2sc(CCCCCCCCC)nc12,0.48200


In [24]:
import os

save_dir = "../data"  # relative dari src/
save_path = os.path.join(save_dir, "augmented_training_data.csv")

df_augmented.to_csv(save_path, index=False)
